In [1]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD')

In [2]:
password = os.environ.get('MYSQL_PASSWORD')

try:
    con = pymysql.connect(
    host='localhost',
    user='root',
    password=password,
    charset='utf8mb4'
    )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")


Connection to the database was successful!


In [3]:
# calling this function makes life easy and just need to pass the name of the database I want to make a query on
# It serves like USE keyword in SQL

def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

In [5]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [42]:
# How to get the names of all the databases in my MySQL server 

# I can use the cursor object from my database connection to execute SQL commands and retrieve information 
# about my databases. Here's an example of how I can use the **SHOW DATABASES** SQL command to retrieve the 
# names of all databases in my MySQL server using my connector function:


# Create a database connection
conn = connector(database='sql_store')

# Create a cursor object
cursor = conn.cursor()

# Execute the SHOW DATABASES command
cursor.execute('SHOW DATABASES')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the names of all databases in the MySQL server
for row in results:
    print(row[0])

# Close the cursor and connection
cursor.close()
conn.close()


information_schema
mysql
performance_schema
sql_hr
sql_inventory
sql_invoicing
sql_store
sys


In [106]:
# How to get the name of the primary keys of tables within a database

# I can use the cursor object from my database connection to execute SQL commands and retrieve 
# information about the primary keys of tables in my database. Here's an example of how I can 
# use the INFORMATION_SCHEMA database to retrieve information about the primary keys of tables in my 
# MySQL server using my connector function:

database = 'sql_store'

# Create a database connection
conn = connector(database)

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL command to retrieve primary key information
cursor.execute('''
    SELECT kcu.table_name, kcu.column_name, kcu.constraint_name
    FROM information_schema.table_constraints tco
    JOIN information_schema.key_column_usage kcu
      ON kcu.constraint_name = tco.constraint_name
      AND kcu.constraint_schema = tco.constraint_schema
      AND kcu.table_name = tco.table_name
    WHERE tco.constraint_type = 'PRIMARY KEY'
      AND tco.table_schema = %s
    ORDER BY kcu.table_name, kcu.ordinal_position;''', (database,))


# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the primary key information
for row in results:
    print(row[0], row[1], row[2])

# Close the cursor and connection
cursor.close()
conn.close()


customers customer_id PRIMARY
order_item_notes note_id PRIMARY
order_items order_id PRIMARY
order_items product_id PRIMARY
order_statuses order_status_id PRIMARY
orders order_id PRIMARY
products product_id PRIMARY
shippers shipper_id PRIMARY


# Retrieving data from a single table

# SELECT

In [8]:
con = connector ('sql_inventory')
df = pd.read_sql("SELECT * FROM products", con)
df


,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [38]:
con = connector ('sql_store')

In [39]:
df = pd.read_sql("SELECT * \
                 FROM customers \
                 WHERE birth_date>'1990-01-01'\
                 ORDER BY last_name", 
                 con)
df


,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [46]:
df = pd.read_sql("SELECT first_name, last_name \
                    FROM customers", 
                    con)
df


,first_name,last_name
0,Babara,MacCaffrey
1,Ines,Brushfield
2,Freddi,Boagey
3,Ambur,Roseburgh
4,Clemmie,Betchley
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Romola,Rumgay
9,Levy,Mynett


In [47]:
df = pd.read_sql("SELECT last_name, first_name \
                    FROM customers", 
                    con)
df


,last_name,first_name
0,MacCaffrey,Babara
1,Brushfield,Ines
2,Boagey,Freddi
3,Roseburgh,Ambur
4,Betchley,Clemmie
5,Twiddell,Elka
6,Dowson,Ilene
7,Naseby,Thacher
8,Rumgay,Romola
9,Mynett,Levy


In [67]:
pd.read_sql("SELECT first_name, last_name, points, points*10 AS '10x POINTS' \
            FROM customers", con)

,first_name,last_name,points,10x POINTS
0,Babara,MacCaffrey,2273,22730
1,Ines,Brushfield,947,9470
2,Freddi,Boagey,2967,29670
3,Ambur,Roseburgh,457,4570
4,Clemmie,Betchley,3675,36750
5,Elka,Twiddell,3073,30730
6,Ilene,Dowson,1672,16720
7,Thacher,Naseby,205,2050
8,Romola,Rumgay,1486,14860
9,Levy,Mynett,796,7960


In [76]:
pd.read_sql("SELECT DISTINCT state FROM customers", con)

,state
0,MA
1,VA
2,CO
3,FL
4,TX
5,IL
6,TN
7,CA
8,GA


In [77]:
con = connector ('sql_inventory')

In [81]:
pd.read_sql("SELECT name, unit_price, unit_price * 1.1 AS 'new price' \
            FROM products", con)

,name,unit_price,new price
0,Foam Dinner Plate,1.21,1.331
1,"Pork - Bacon,back Peameal",4.65,5.115
2,"Lettuce - Romaine, Heart",3.35,3.685
3,"Brocolinni - Gaylan, Chinese",4.53,4.983
4,Sauce - Ranch Dressing,1.63,1.793
5,Petit Baguette,2.39,2.629
6,Sweet Pea Sprouts,3.29,3.619
7,Island Oasis - Raspberry,0.74,0.814
8,Longan,2.26,2.486
9,Broom - Push,1.09,1.199


In [82]:
con = connector('sql_store')

# WHERE

In [83]:
pd.read_sql("SELECT * FROM customers WHERE points>3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [87]:
pd.read_sql('SELECT * FROM customers WHERE state != "VA"', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [89]:
pd.read_sql("SELECT * FROM customers WHERE birth_date > '1990-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [3]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [98]:
pd.read_sql("SELECT * FROM orders WHERE order_date>='2019-01-01'", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None


In [6]:
con = connector('sql_store')

# AND OR NOT

In [13]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' AND points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [14]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
6,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [17]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR \
            (points>1000 AND state ='VA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [21]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE NOT (birth_date > '1990-01-01' OR points>1000)", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [23]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE  birth_date <= '1990-01-01' AND points<=1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
con = connector('sql_store')

pd.read_sql("SELECT * \
            FROM order_items \
            WHERE (order_id = 6 AND (unit_price * quantity)> 30)", con)

,order_id,product_id,quantity,unit_price
0,6,1,4,8.65


In [34]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state ='VA' OR state = 'GA' OR state = 'FL'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# IN

In [35]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [36]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state NOT IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [39]:
pd.read_sql("SELECT *\
            FROM products\
            WHERE quantity_in_stock IN (49, 38, 72)", con)

,product_id,name,quantity_in_stock,unit_price
0,2,"Pork - Bacon,back Peameal",49,4.65
1,3,"Lettuce - Romaine, Heart",38,3.35


In [40]:
pd.read_sql("SELECT *\
            FROM customers\
                WHERE points >=1000 AND points <=3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# BETWEEN

In [43]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE points BETWEEN 1000 AND 3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [48]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE birth_date BETWEEN '1990-01-01' AND '2000-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# LIKE

In [49]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [50]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'brush%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [51]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [52]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points


In [55]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [56]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [69]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE address LIKE '%TRAIL%' OR \
                  address LIKE '%AVENUE%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [70]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [71]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone NOT LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
6,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [4]:
con = connector('sql_store')

In [5]:
pd.read_sql('SELECT * FROM customers', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# REGEXP

In [6]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%field%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [7]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [8]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^Brush'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [9]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [14]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [15]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [16]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [17]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [23]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[gim]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [25]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'e[l]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [26]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[a-h]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [27]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE first_name REGEXP 'ELKA|AMBUR'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [28]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'EY$|ON$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [29]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^MY|SE'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
pd.read_sql("SELECT * \
            FROM customers\
            WHERE last_name REGEXP 'B[RU]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


# IS NULL

In [35]:
pd.read_sql("SELECT * \
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [38]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [39]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NOT NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [40]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [48]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [52]:
pd.read_sql("SELECT *\
            FROM orders\
            WHERE shipped_date IS NULL", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None
1,3,8,2017-12-01,1,None,None,None
2,4,2,2017-01-22,1,None,None,None
3,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,None
4,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,None


# ORDER BY

In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [54]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name DESC", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
3,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
6,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
7,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
8,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
9,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [55]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
7,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
8,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
9,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [56]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state DESC, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
6,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
9,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [58]:
pd.read_sql("SELECT first_name, last_name\
            FROM customers\
            ORDER BY birth_date", con)

,first_name,last_name
0,Ilene,Dowson
1,Levy,Mynett
2,Clemmie,Betchley
3,Ambur,Roseburgh
4,Freddi,Boagey
5,Babara,MacCaffrey
6,Ines,Brushfield
7,Elka,Twiddell
8,Romola,Rumgay
9,Thacher,Naseby


In [60]:
pd.read_sql("SELECT first_name, last_name, 10 AS points\
            FROM customers\
            ORDER BY points, birth_date", con)

,first_name,last_name,points
0,Ilene,Dowson,10
1,Levy,Mynett,10
2,Clemmie,Betchley,10
3,Ambur,Roseburgh,10
4,Freddi,Boagey,10
5,Babara,MacCaffrey,10
6,Ines,Brushfield,10
7,Elka,Twiddell,10
8,Romola,Rumgay,10
9,Thacher,Naseby,10


In [72]:
pd.read_sql("SELECT *, quantity * unit_price AS total_price\
            FROM order_items\
            WHERE order_id = 2\
            ORDER BY total_price DESC", con)

,order_id,product_id,quantity,unit_price,total_price
0,2,1,2,9.10,18.20
1,2,4,4,1.66,6.64
2,2,6,2,2.94,5.88


# LIMIT

In [75]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 4", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [76]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 400", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [79]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 6, 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [82]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY points DESC\
            LIMIT 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


# Retrieving data from multiple tables

# INNER JOIN

In [7]:
con = connector('sql_store')

In [8]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [10]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [12]:
pd.read_sql("SELECT *\
            FROM orders\
            JOIN customers \
            ON orders.customer_id = customers.customer_id", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,6,2019-01-30,1,None,None,NaN,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,2,7,2018-08-02,2,None,2018-08-03,4.0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
2,3,8,2017-12-01,1,None,None,NaN,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,4,2,2017-01-22,1,None,None,NaN,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
4,5,5,2017-08-25,2,,2017-08-26,3.0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
6,7,2,2018-09-22,2,None,2018-09-23,4.0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
9,10,6,2018-04-22,2,None,2018-04-23,2.0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [15]:
pd.read_sql("SELECT order_id, orders.customer_id, first_name, last_name\
            FROM orders\
            JOIN customers \
                ON orders.customer_id = customers.customer_id", con)

,order_id,customer_id,first_name,last_name
0,4,2,Ines,Brushfield
1,7,2,Ines,Brushfield
2,5,5,Clemmie,Betchley
3,8,5,Clemmie,Betchley
4,1,6,Elka,Twiddell
5,10,6,Elka,Twiddell
6,2,7,Ilene,Dowson
7,3,8,Thacher,Naseby
8,6,10,Levy,Mynett
9,9,10,Levy,Mynett


In [19]:
pd.read_sql("SELECT order_id, o.customer_id, first_name, last_name\
            FROM orders o\
            JOIN customers c \
                ON o.customer_id = c.customer_id", con)

,order_id,customer_id,first_name,last_name
0,4,2,Ines,Brushfield
1,7,2,Ines,Brushfield
2,5,5,Clemmie,Betchley
3,8,5,Clemmie,Betchley
4,1,6,Elka,Twiddell
5,10,6,Elka,Twiddell
6,2,7,Ilene,Dowson
7,3,8,Thacher,Naseby
8,6,10,Levy,Mynett
9,9,10,Levy,Mynett


In [22]:
pd.read_sql("SELECT *\
            FROM order_items", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


In [23]:
pd.read_sql("SELECT *\
            FROM products", con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [30]:
pd.read_sql("SELECT order_id, oi.product_id, quantity, oi.unit_price\
            FROM order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price
0,2,1,2,9.10
1,6,1,4,8.65
2,10,1,10,6.01
3,5,2,3,9.89
4,6,2,4,3.28
5,3,3,10,9.12
6,4,3,7,6.99
7,6,3,4,7.46
8,7,3,7,9.17
9,1,4,4,3.74


In [31]:
pd.read_sql("SELECT order_id, oi.product_id, quantity, oi.unit_price\
            FROM order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id\
                LIMIT 6, 3", con)

,order_id,product_id,quantity,unit_price
0,4,3,7,6.99
1,6,3,4,7.46
2,7,3,7,9.17


# JOINING ACROSS DATABASES

In [43]:

# Create a database connection
conn = connector(database='sql_store')

# Create a cursor object
cursor = conn.cursor()

# Execute the SHOW DATABASES command
cursor.execute('SHOW DATABASES')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the names of all databases in the MySQL server
for row in results:
    print(row[0])

# Close the cursor and connection
cursor.close()
conn.close()


information_schema
mysql
performance_schema
sql_hr
sql_inventory
sql_invoicing
sql_store
sys


In [44]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN sql_inventory.products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price,product_id,name,quantity_in_stock,unit_price
0,2,1,2,9.10,1,Foam Dinner Plate,70,1.21
1,6,1,4,8.65,1,Foam Dinner Plate,70,1.21
2,10,1,10,6.01,1,Foam Dinner Plate,70,1.21
3,5,2,3,9.89,2,"Pork - Bacon,back Peameal",49,4.65
4,6,2,4,3.28,2,"Pork - Bacon,back Peameal",49,4.65
5,3,3,10,9.12,3,"Lettuce - Romaine, Heart",38,3.35
6,4,3,7,6.99,3,"Lettuce - Romaine, Heart",38,3.35
7,6,3,4,7.46,3,"Lettuce - Romaine, Heart",38,3.35
8,7,3,7,9.17,3,"Lettuce - Romaine, Heart",38,3.35
9,1,4,4,3.74,4,"Brocolinni - Gaylan, Chinese",90,4.53


In [45]:
con = connector('sql_inventory')

pd.read_sql("SELECT *\
            FROM sql_store.order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price,product_id,name,quantity_in_stock,unit_price
0,2,1,2,9.10,1,Foam Dinner Plate,70,1.21
1,6,1,4,8.65,1,Foam Dinner Plate,70,1.21
2,10,1,10,6.01,1,Foam Dinner Plate,70,1.21
3,5,2,3,9.89,2,"Pork - Bacon,back Peameal",49,4.65
4,6,2,4,3.28,2,"Pork - Bacon,back Peameal",49,4.65
5,3,3,10,9.12,3,"Lettuce - Romaine, Heart",38,3.35
6,4,3,7,6.99,3,"Lettuce - Romaine, Heart",38,3.35
7,6,3,4,7.46,3,"Lettuce - Romaine, Heart",38,3.35
8,7,3,7,9.17,3,"Lettuce - Romaine, Heart",38,3.35
9,1,4,4,3.74,4,"Brocolinni - Gaylan, Chinese",90,4.53


# SELF JOIN

In [46]:
con = connector('sql_hr')

In [56]:
pd.read_sql("SELECT *\
            FROM employees\
            LIMIT 7", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1
3,40448,Mindy,Crissil,Staff Scientist,94860,37270.0,1
4,56274,Keriann,Alloisi,VP Marketing,110150,37270.0,1
5,63196,Alaster,Scutchin,Assistant Professor,32179,37270.0,2
6,67009,North,de Clerc,VP Product Management,114257,37270.0,2


In [55]:
pd.read_sql("SELECT *\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
1,37851,Sayer,Matterson,Statistician III,98926,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
2,40448,Mindy,Crissil,Staff Scientist,94860,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
3,56274,Keriann,Alloisi,VP Marketing,110150,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
4,63196,Alaster,Scutchin,Assistant Professor,32179,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
5,67009,North,de Clerc,VP Product Management,114257,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
6,67370,Elladine,Rising,Social Worker,96767,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
7,68249,Nisse,Voysey,Financial Advisor,52832,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
8,72540,Guthrey,Iacopetti,Office Assistant I,117690,37270,3,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
9,72913,Kass,Hefferan,Computer Systems Analyst IV,96401,37270,3,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10


In [57]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name,\
                m.first_name AS 'Manager first name'\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,first_name,Manager first name
0,33391,D'arcy,Yovonnda
1,37851,Sayer,Yovonnda
2,40448,Mindy,Yovonnda
3,56274,Keriann,Yovonnda
4,63196,Alaster,Yovonnda
5,67009,North,Yovonnda
6,67370,Elladine,Yovonnda
7,68249,Nisse,Yovonnda
8,72540,Guthrey,Yovonnda
9,72913,Kass,Yovonnda


# JOINING MULTIPLE TABLES

In [58]:
con = connector('sql_store')

In [60]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [59]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [61]:
pd.read_sql("SELECT *\
            FROM order_statuses", con)

,order_status_id,name
0,1,Processed
1,2,Shipped
2,3,Delivered


In [62]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [63]:
# join orders table with 2 tables: customers table and order_statuses tables

In [66]:
pd.read_sql("SELECT *\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id,customer_id,first_name,last_name,birth_date,phone,address,city,state,points,order_status_id,name
0,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675,1,Processed
1,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796,1,Processed
2,4,2,2017-01-22,1,None,None,NaN,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947,1,Processed
3,3,8,2017-12-01,1,None,None,NaN,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205,1,Processed
4,1,6,2019-01-30,1,None,None,NaN,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073,1,Processed
5,10,6,2018-04-22,2,None,2018-04-23,2.0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073,2,Shipped
6,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796,2,Shipped
7,7,2,2018-09-22,2,None,2018-09-23,4.0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947,2,Shipped
8,5,5,2017-08-25,2,,2017-08-26,3.0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675,2,Shipped
9,2,7,2018-08-02,2,None,2018-08-03,4.0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672,2,Shipped


In [72]:
pd.read_sql("SELECT \
                o.order_id,\
                o.order_date,\
                c.first_name,\
                c.last_name,\
                os.name AS status\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,order_date,first_name,last_name,status
0,8,2018-06-08,Clemmie,Betchley,Processed
1,6,2018-11-18,Levy,Mynett,Processed
2,4,2017-01-22,Ines,Brushfield,Processed
3,3,2017-12-01,Thacher,Naseby,Processed
4,1,2019-01-30,Elka,Twiddell,Processed
5,10,2018-04-22,Elka,Twiddell,Shipped
6,9,2017-07-05,Levy,Mynett,Shipped
7,7,2018-09-22,Ines,Brushfield,Shipped
8,5,2017-08-25,Clemmie,Betchley,Shipped
9,2,2018-08-02,Ilene,Dowson,Shipped


In [73]:
con = connector('sql_invoicing')

In [74]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_invoicing';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [76]:
pd.read_sql("SELECT *\
            FROM clients", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [77]:
pd.read_sql("SELECT *\
            FROM payment_methods", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal
3,4,Wire Transfer


In [78]:
pd.read_sql("SELECT *\
            FROM payments", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [87]:
pd.read_sql("SELECT \
                p.date,\
                p.invoice_id,\
                p.amount,\
                c.name,\
                pm.name AS 'Payment method'\
            FROM clients c\
            JOIN payments p\
                ON c.client_id = p.client_id\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,invoice_id,amount,name,Payment method
0,2019-02-12,2,8.18,Topiclounge,Credit Card
1,2019-01-03,6,74.55,Vinte,Credit Card
2,2019-01-11,11,0.03,Yadel,Credit Card
3,2019-01-26,13,87.44,Topiclounge,Credit Card
4,2019-01-15,15,80.31,Yadel,Credit Card
5,2019-01-15,17,68.10,Yadel,Credit Card
6,2019-01-08,18,32.77,Topiclounge,Credit Card
7,2019-01-08,18,10.00,Topiclounge,Cash


# COMPOUND JOIN CONDITIONS

In [140]:
# let's get the primary keys of each table

database = 'sql_store'

# Create a database connection
conn = connector(database)

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL command to retrieve primary key information
cursor.execute('''
    SELECT kcu.table_name, kcu.column_name, kcu.constraint_name
    FROM information_schema.table_constraints tco
    JOIN information_schema.key_column_usage kcu
      ON kcu.constraint_name = tco.constraint_name
      AND kcu.constraint_schema = tco.constraint_schema
      AND kcu.table_name = tco.table_name
    WHERE tco.constraint_type = 'PRIMARY KEY'
      AND tco.table_schema = %s
    ORDER BY kcu.table_name, kcu.ordinal_position;''', (database,))


# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the primary key information
for row in results:
    print(row[0], row[1], row[2])

# Close the cursor and connection
cursor.close()
conn.close()


customers customer_id PRIMARY
order_item_notes note_id PRIMARY
order_items order_id PRIMARY
order_items product_id PRIMARY
order_statuses order_status_id PRIMARY
orders order_id PRIMARY
products product_id PRIMARY
shippers shipper_id PRIMARY


In [136]:
con = connector('sql_store')

In [133]:
pd.read_sql("SELECT *\
            FROM order_items", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


In [142]:
pd.read_sql("SELECT *\
            FROM order_item_notes", con)

,note_id,order_Id,product_id,note
0,1,2,1,first note
1,2,2,1,second note


In [141]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.10,1,2,1,first note
1,2,4,4,1.66,1,2,1,first note
2,2,6,2,2.94,1,2,1,first note
3,2,1,2,9.10,2,2,1,second note
4,2,4,4,1.66,2,2,1,second note
5,2,6,2,2.94,2,2,1,second note


In [138]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id\
                AND oi.product_id = oin.product_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.1,1,2,1,first note
1,2,1,2,9.1,2,2,1,second note


# IMPLICIT JOIN SYNTAX 
DON'T USE IT SINCE IF YOU FORGET **WHERE CLAUSE** YOU GET CROSS JOIN SO USE EXPLICIT JOIN INSTEAD.

In [147]:
# THIS IS THE EXPLICIT JOIN, AS BEFORE
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id", con)

,first_name,last_name
0,Ines,Brushfield
1,Ines,Brushfield
2,Clemmie,Betchley
3,Clemmie,Betchley
4,Elka,Twiddell
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Levy,Mynett
9,Levy,Mynett


In [148]:
# THIS IS IMP[LICIT JOIN]
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o, customers c\
            WHERE o.customer_id = c.customer_id", con)

,first_name,last_name
0,Ines,Brushfield
1,Ines,Brushfield
2,Clemmie,Betchley
3,Clemmie,Betchley
4,Elka,Twiddell
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Levy,Mynett
9,Levy,Mynett


In [149]:
# HERE IS WHAT YOU GET IF YOU FORGET WHERE CLAUSE
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o, customers c", con)

,first_name,last_name
0,Babara,MacCaffrey
1,Babara,MacCaffrey
2,Babara,MacCaffrey
3,Babara,MacCaffrey
4,Babara,MacCaffrey
...,...,...
95,Levy,Mynett
96,Levy,Mynett
97,Levy,Mynett
98,Levy,Mynett


# OUTER JOINS : LEFT JOIN and RIGHT JOIN

In [165]:
# this is inner join
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,2,Ines,4
1,2,Ines,7
2,5,Clemmie,5
3,5,Clemmie,8
4,6,Elka,1
5,6,Elka,10
6,7,Ilene,2
7,8,Thacher,3
8,10,Levy,6
9,10,Levy,9


In [166]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,1,Babara,NaN
1,2,Ines,4.0
2,2,Ines,7.0
3,3,Freddi,NaN
4,4,Ambur,NaN
5,5,Clemmie,5.0
6,5,Clemmie,8.0
7,6,Elka,1.0
8,6,Elka,10.0
9,7,Ilene,2.0


In [167]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            RIGHT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,2,Ines,4
1,2,Ines,7
2,5,Clemmie,5
3,5,Clemmie,8
4,6,Elka,1
5,6,Elka,10
6,7,Ilene,2
7,8,Thacher,3
8,10,Levy,6
9,10,Levy,9


In [168]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM orders o\
            RIGHT JOIN customers c\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,1,Babara,NaN
1,2,Ines,4.0
2,2,Ines,7.0
3,3,Freddi,NaN
4,4,Ambur,NaN
5,5,Clemmie,5.0
6,5,Clemmie,8.0
7,6,Elka,1.0
8,6,Elka,10.0
9,7,Ilene,2.0


In [158]:
pd.read_sql("SELECT \
                p.product_id,\
                p.name,\
                oi.quantity\
            FROM products p\
            LEFT JOIN order_items oi\
                ON p.product_id = oi.product_id", con)

,product_id,name,quantity
0,1,Foam Dinner Plate,2.0
1,1,Foam Dinner Plate,4.0
2,1,Foam Dinner Plate,10.0
3,2,"Pork - Bacon,back Peameal",3.0
4,2,"Pork - Bacon,back Peameal",4.0
5,3,"Lettuce - Romaine, Heart",10.0
6,3,"Lettuce - Romaine, Heart",7.0
7,3,"Lettuce - Romaine, Heart",4.0
8,3,"Lettuce - Romaine, Heart",7.0
9,4,"Brocolinni - Gaylan, Chinese",4.0


# OUTER JOINS BETWEEN MULTIPLE TABLES

In [4]:
con = connector('sql_store')

In [9]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id,\
                o.shipper_id\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id,shipper_id
0,1,Babara,NaN,NaN
1,2,Ines,7.0,4.0
2,2,Ines,4.0,NaN
3,3,Freddi,NaN,NaN
4,4,Ambur,NaN,NaN
5,5,Clemmie,8.0,NaN
6,5,Clemmie,5.0,3.0
7,6,Elka,10.0,2.0
8,6,Elka,1.0,NaN
9,7,Ilene,2.0,4.0


In [10]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [7]:
pd.read_sql("SELECT *\
            FROM shippers", con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"


In [20]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id,\
                s.name AS shipper_name\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            LEFT JOIN shippers s\
                ON o.shipper_id = s.shipper_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id,shipper_name
0,1,Babara,NaN,None
1,2,Ines,4.0,None
2,2,Ines,7.0,"Mraz, Renner and Nolan"
3,3,Freddi,NaN,None
4,4,Ambur,NaN,None
5,5,Clemmie,5.0,Satterfield LLC
6,5,Clemmie,8.0,None
7,6,Elka,1.0,None
8,6,Elka,10.0,Schinner-Predovic
9,7,Ilene,2.0,"Mraz, Renner and Nolan"


In [21]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [22]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [23]:
pd.read_sql("SELECT *\
            FROM order_statuses", con)

,order_status_id,name
0,1,Processed
1,2,Shipped
2,3,Delivered


In [24]:
pd.read_sql("SELECT *\
            FROM shippers", con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"


In [36]:
pd.read_sql("SELECT \
                o.order_id,\
                o.order_date,\
                c.first_name,\
                s.name AS shipper,\
                os.name AS status\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            LEFT JOIN shippers s\
                ON o.shipper_id = s.shipper_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,order_date,first_name,shipper,status
0,8,2018-06-08,Clemmie,None,Processed
1,6,2018-11-18,Levy,None,Processed
2,4,2017-01-22,Ines,None,Processed
3,3,2017-12-01,Thacher,None,Processed
4,1,2019-01-30,Elka,None,Processed
5,10,2018-04-22,Elka,Schinner-Predovic,Shipped
6,9,2017-07-05,Levy,Hettinger LLC,Shipped
7,7,2018-09-22,Ines,"Mraz, Renner and Nolan",Shipped
8,5,2017-08-25,Clemmie,Satterfield LLC,Shipped
9,2,2018-08-02,Ilene,"Mraz, Renner and Nolan",Shipped


# SELF OUTER JOIN

In [38]:
con = connector('sql_hr')

In [40]:
pd.read_sql('SELECT *\
            FROM employees\
            LIMIT 3', con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1


In [43]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name AS employee,\
                m.first_name AS manager\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,employee,manager
0,33391,D'arcy,Yovonnda
1,37851,Sayer,Yovonnda
2,40448,Mindy,Yovonnda
3,56274,Keriann,Yovonnda
4,63196,Alaster,Yovonnda
5,67009,North,Yovonnda
6,67370,Elladine,Yovonnda
7,68249,Nisse,Yovonnda
8,72540,Guthrey,Yovonnda
9,72913,Kass,Yovonnda


In [44]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name AS employee,\
                m.first_name AS manager\
            FROM employees e\
            LEFT JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,employee,manager
0,33391,D'arcy,Yovonnda
1,37270,Yovonnda,None
2,37851,Sayer,Yovonnda
3,40448,Mindy,Yovonnda
4,56274,Keriann,Yovonnda
5,63196,Alaster,Yovonnda
6,67009,North,Yovonnda
7,67370,Elladine,Yovonnda
8,68249,Nisse,Yovonnda
9,72540,Guthrey,Yovonnda


# USING CLAUSE

In [45]:
con = connector('sql_store')

In [52]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


In [53]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


In [56]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name,\
                s.name AS shipper\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)\
            JOIN shippers s\
                USING (shipper_id)", con)

,order_id,first_name,shipper
0,9,Levy,Hettinger LLC
1,10,Elka,Schinner-Predovic
2,5,Clemmie,Satterfield LLC
3,2,Ilene,"Mraz, Renner and Nolan"
4,7,Ines,"Mraz, Renner and Nolan"


In [57]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name,\
                s.name AS shipper\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)\
            LEFT JOIN shippers s\
                USING (shipper_id)", con)

,order_id,first_name,shipper
0,1,Elka,None
1,2,Ilene,"Mraz, Renner and Nolan"
2,3,Thacher,None
3,4,Ines,None
4,5,Clemmie,Satterfield LLC
5,6,Levy,None
6,7,Ines,"Mraz, Renner and Nolan"
7,8,Clemmie,None
8,9,Levy,Hettinger LLC
9,10,Elka,Schinner-Predovic


In [58]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id AND\
                    oi.product_id = oin.product_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.1,1,2,1,first note
1,2,1,2,9.1,2,2,1,second note


In [59]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                USING (order_id, product_id)", con)

,order_id,product_id,quantity,unit_price,note_id,note
0,2,1,2,9.1,1,first note
1,2,1,2,9.1,2,second note


In [60]:
con = connector('sql_invoicing')

In [64]:
pd.read_sql("SELECT *\
            FROM clients", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [66]:
pd.read_sql("SELECT *\
            FROM payments", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [68]:
pd.read_sql("SELECT *\
            FROM payment_methods", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal
3,4,Wire Transfer


In [73]:
pd.read_sql("SELECT \
            p.date,\
            c.name AS client,\
            p.amount,\
            pm.name AS payment_method\
            FROM payments p\
            JOIN clients c\
                ON p.client_id = c.client_id\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,client,amount,payment_method
0,2019-02-12,Topiclounge,8.18,Credit Card
1,2019-01-03,Vinte,74.55,Credit Card
2,2019-01-11,Yadel,0.03,Credit Card
3,2019-01-26,Topiclounge,87.44,Credit Card
4,2019-01-15,Yadel,80.31,Credit Card
5,2019-01-15,Yadel,68.10,Credit Card
6,2019-01-08,Topiclounge,32.77,Credit Card
7,2019-01-08,Topiclounge,10.00,Cash


In [72]:
pd.read_sql("SELECT \
            p.date,\
            c.name AS client,\
            p.amount,\
            pm.name AS payment_method\
            FROM payments p\
            JOIN clients c\
                USING (client_id)\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,client,amount,payment_method
0,2019-02-12,Topiclounge,8.18,Credit Card
1,2019-01-03,Vinte,74.55,Credit Card
2,2019-01-11,Yadel,0.03,Credit Card
3,2019-01-26,Topiclounge,87.44,Credit Card
4,2019-01-15,Yadel,80.31,Credit Card
5,2019-01-15,Yadel,68.10,Credit Card
6,2019-01-08,Topiclounge,32.77,Credit Card
7,2019-01-08,Topiclounge,10.00,Cash


# NATURAL JOINS -- not recommended due to producing unexpected results

In [78]:
con = connector('sql_store')

In [86]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            NATURAL JOIN customers c", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


# CROSS JOIN

In [91]:
pd.read_sql("SELECT \
                c.first_name AS customer,\
                p.name AS product\
            FROM customers c\
            CROSS JOIN products p\
            ORDER BY c.first_name", con)

,customer,product
0,Ambur,Sweet Pea Sprouts
1,Ambur,Island Oasis - Raspberry
2,Ambur,"Lettuce - Romaine, Heart"
3,Ambur,Petit Baguette
4,Ambur,Foam Dinner Plate
...,...,...
95,Thacher,Petit Baguette
96,Thacher,"Pork - Bacon,back Peameal"
97,Thacher,Sweet Pea Sprouts
98,Thacher,Foam Dinner Plate


In [92]:
# the implicit syntax for cross join is as follows (but you use explicit syntax as above for clarity)

pd.read_sql("SELECT \
                c.first_name AS customer,\
                p.name AS product\
            FROM customers c, products p\
            ORDER BY c.first_name", con)

,customer,product
0,Ambur,Sweet Pea Sprouts
1,Ambur,Island Oasis - Raspberry
2,Ambur,"Lettuce - Romaine, Heart"
3,Ambur,Petit Baguette
4,Ambur,Foam Dinner Plate
...,...,...
95,Thacher,Petit Baguette
96,Thacher,"Pork - Bacon,back Peameal"
97,Thacher,Sweet Pea Sprouts
98,Thacher,Foam Dinner Plate


In [95]:
pd.read_sql("SELECT \
                s.name AS shipper,\
                p.name AS product\
            FROM shippers s, products p\
            ORDER BY s.name", con)

,shipper,product
0,Hettinger LLC,Sweet Pea Sprouts
1,Hettinger LLC,"Brocolinni - Gaylan, Chinese"
2,Hettinger LLC,"Lettuce - Romaine, Heart"
3,Hettinger LLC,Sauce - Ranch Dressing
4,Hettinger LLC,Petit Baguette
5,Hettinger LLC,"Pork - Bacon,back Peameal"
6,Hettinger LLC,Island Oasis - Raspberry
7,Hettinger LLC,Foam Dinner Plate
8,Hettinger LLC,Longan
9,Hettinger LLC,Broom - Push


In [96]:
pd.read_sql("SELECT \
                s.name AS shipper,\
                p.name AS product\
            FROM shippers s\
            CROSS JOIN products p\
            ORDER BY s.name", con)

,shipper,product
0,Hettinger LLC,Sweet Pea Sprouts
1,Hettinger LLC,"Brocolinni - Gaylan, Chinese"
2,Hettinger LLC,"Lettuce - Romaine, Heart"
3,Hettinger LLC,Sauce - Ranch Dressing
4,Hettinger LLC,Petit Baguette
5,Hettinger LLC,"Pork - Bacon,back Peameal"
6,Hettinger LLC,Island Oasis - Raspberry
7,Hettinger LLC,Foam Dinner Plate
8,Hettinger LLC,Longan
9,Hettinger LLC,Broom - Push


# UNIONS

In [110]:
pd.read_sql("SELECT \
                order_id,\
                order_date,\
                'Active' AS status\
            FROM orders\
            WHERE order_date >= '2019-01-01'\
            UNION \
            SELECT \
                order_id,\
                order_date,\
                'Archived' AS status\
            FROM orders\
            WHERE order_date <'2019-01-01'\
            ", con)

,order_id,order_date,status
0,1,2019-01-30,Active
1,2,2018-08-02,Archived
2,3,2017-12-01,Archived
3,4,2017-01-22,Archived
4,5,2017-08-25,Archived
5,6,2018-11-18,Archived
6,7,2018-09-22,Archived
7,8,2018-06-08,Archived
8,9,2017-07-05,Archived
9,10,2018-04-22,Archived


In [125]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Bronze' AS type\
            FROM customers c\
            WHERE c.points <= 2000\
            UNION\
            SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Silver' AS type\
            FROM customers c\
            WHERE c.points BETWEEN 2000 AND 3000\
            UNION\
            SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Gold' AS type\
            FROM customers c\
            WHERE c.points >= 3000\
            ORDER BY first_name", con)

,customer_id,first_name,points,type
0,4,Ambur,457,Bronze
1,1,Babara,2273,Silver
2,5,Clemmie,3675,Gold
3,6,Elka,3073,Gold
4,3,Freddi,2967,Silver
5,7,Ilene,1672,Bronze
6,2,Ines,947,Bronze
7,10,Levy,796,Bronze
8,9,Romola,1486,Bronze
9,8,Thacher,205,Bronze


In [127]:
# no need to give alias to the table name as above so i repeat it in the following: 

pd.read_sql("SELECT \
                customer_id,\
                first_name,\
                points,\
                'Bronze' AS type\
            FROM customers \
            WHERE points <= 2000\
            UNION\
            SELECT \
                customer_id,\
                first_name,\
                points,\
                'Silver' AS type\
            FROM customers \
            WHERE points BETWEEN 2000 AND 3000\
            UNION\
            SELECT \
                customer_id,\
                first_name,\
                points,\
                'Gold' AS type\
            FROM customers \
            WHERE points >= 3000\
            ORDER BY first_name", con)

,customer_id,first_name,points,type
0,4,Ambur,457,Bronze
1,1,Babara,2273,Silver
2,5,Clemmie,3675,Gold
3,6,Elka,3073,Gold
4,3,Freddi,2967,Silver
5,7,Ilene,1672,Bronze
6,2,Ines,947,Bronze
7,10,Levy,796,Bronze
8,9,Romola,1486,Bronze
9,8,Thacher,205,Bronze


# INSERTING - UPDATING - DELETING DATA